# London Crime Conv1D Classifier
Predicting **Crime type** using TensorFlow/Keras Conv1D Neural Network


In [2]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical


2025-12-04 17:50:14.207079: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-04 17:50:14.207473: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 17:50:14.260582: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 17:50:17.236139: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

## Load Dataset

In [3]:
df = pd.read_csv('2022_2025_city_of_london_street.csv')
df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,source_file,source_folder
0,5a10b2cf8fa37fbf3933f66319f1c2ce76ef9697e19442...,2024-12,City of London Police,City of London Police,-0.105965,51.518514,On or near Further/Higher Educational Building,E01000916,Camden 027B,Robbery,Investigation complete; no suspect identified,NaN,2024-12-city-of-london-street.csv,datasets
1,71ac2a8e6b747e5044b1bb5e8b93b8b13a09bf39649390...,2024-12,City of London Police,City of London Police,-0.111596,51.518281,On or near Chancery Lane,E01000914,Camden 028B,Other theft,Investigation complete; no suspect identified,NaN,2024-12-city-of-london-street.csv,datasets
2,e5dc3c4e04ebc5285c57dabef730c271df889665252ed9...,2024-12,City of London Police,City of London Police,-0.112096,51.515942,On or near Nightclub,E01000914,Camden 028B,Robbery,Status update unavailable,NaN,2024-12-city-of-london-street.csv,datasets
3,2072389e40234ffec9830716e10ba0aa4ad91fbe282bf5...,2024-12,City of London Police,City of London Police,-0.112537,51.519582,On or near Gray'S Inn Square,E01000914,Camden 028B,Shoplifting,Status update unavailable,NaN,2024-12-city-of-london-street.csv,datasets
4,5fe3737ddf7156d6fb68bb3a2b54dcb1f7bc354f5f70e9...,2024-12,City of London Police,City of London Police,-0.111596,51.518281,On or near Chancery Lane,E01000914,Camden 028B,Theft from the person,Investigation complete; no suspect identified,NaN,2024-12-city-of-london-street.csv,datasets


## Data Preprocessing

In [ ]:
# Drop duplicates rows
df.drop_duplicates(inplace=True)

# Drop the Crime ID, source column, and source file columns since they contain duplicate information/administrative purposes when merging the datasets, or are identifiers.
df.drop(columns=['Crime ID', 'Source', 'Source file'], inplace=True)

# Strip column names
# REASON: In some cases, when reading CSV files, extra spaces can be inadvertently added in the column names.
# This can lead to issues when trying to access these columns later in the code, as the names won't match exactly.
df.columns = df.columns.str.strip()

# Fill missing numeric values using KNN imputation, which in this case is the GPS coordinates (Longitude, Latitude)
numeric_cols = ['Longitude', 'Latitude']
imputer = KNNImputer(n_neighbors=5)
# Note: For production, fit the imputer on the training set only to avoid data leakage.
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Encode target values, so that the crime ('Crime type') can be used for classification, (LABELS) to (NUMERICAL RANGE)
target_col = 'Crime type'
encoder = LabelEncoder()

df['target_encoded'] = encoder.fit_transform(df[target_col])
num_classes = df['target_encoded'].nunique()

# Features: numeric and categorical
categorical_cols = ['Reported by', 'Falls within', 'LSOA code', 'LSOA name']
df_encoded = pd.get_dummies(df[categorical_cols])

X = pd.concat([df[numeric_cols], df_encoded], axis=1).values
y = to_categorical(df['target_encoded'], num_classes=num_classes)

# Standardize numeric features
scaler = StandardScaler()
X[:, :len(numeric_cols)] = scaler.fit_transform(X[:, :len(numeric_cols)])

# Reshape for Conv1D (samples, timesteps, features=1)
X = X.reshape((X.shape[0], X.shape[1], 1))


## Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Model Definition

In [ ]:
input_shape = X_train.shape[1:]

model = Sequential([
    Conv1D(64, kernel_size=3, padding='same', activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Conv1D(256, kernel_size=3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## Training

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=512
)

## Evaluation

In [ ]:
# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification Report
print(classification_report(y_true_classes, y_pred_classes, target_names=encoder.classes_))

## Plot Training Curves

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend(); plt.title('Loss')

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend(); plt.title('Accuracy')

plt.show()